In [ ]:
import sys
import os
import gc
import collections

import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams, cycler
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.signal import butter, lfilter, freqz, bessel
from tqdm import tqdm
import iaaft

import numpy as np
import xarray as xr
import geopandas as gpd
import rioxarray
from xeofs.xarray import EOF

import statsmodels.api as sm
import scipy
from scipy import signal
from sklearn.metrics import mean_squared_error
from itertools import product
import pyproj
from shapely.geometry import mapping
from xarrayutils.utils import linear_trend, xr_linregress
import pandas as pd
import cmocean

# NOTE: Local xeofs to be submitted as pull-request (addition of xeofs.model.reconstruct_randomized_X method)

In [ ]:
# Define local plotting parameters
#sns.set_theme(style="whitegrid")
#sns.set_theme(style="ticks")
sns.set_theme(style="white")
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
plt.rcParams.update({'font.size': 15})
plt.rc('font', family='sans-serif') 
plt.rc('font', serif='Helvetica Neue')
plt.rc('text', usetex=True)

## Define directories

In [ ]:
# To update following with relative repository paths once data and code is up on Zenodo
# Current version uses the project template on Github.

# Define project repo path
inDirName = '/Users/smurugan9/research/aislens/aislens_emulation/'

# DATA FILE PATHS
# Data containing regridded flux and SSH for 150 years
regriddedFluxSSH_filepath = 'data/interim/RegriddedFluxSSH.nc'
# File contains all defined ice shelves
iceShelvesShape_filepath = 'data/interim/iceShelves.geojson'

# Relative directory paths for Data and Figures
figures_folderpath = 'reports/figures/'
interim_data_folder = 'data/interim/'
processed_data_folder = 'data/processed/'
flux_dedrafted_data_path = 'dedrafted_flux_IMBIE/'
randomized_realizations_path = 'randomized_realizations/'
flux_dedrafted_iceshelves_data_path = 'iceshelves_dedrafted_flux/'
reconstructions_neofs_path = 'reconstructions_neofs/'
cise_file_path = 'cise_data/'
std_file_path = 'standardized_rec_data/'


pca_1 = '1_pca/'
pca_2 = '2_pca_normalized/'
pca_3 = '3_pca_normalized_nmodes/'
pca_4 = '4_lowpass_ pca_normalized/'

## Input data files and catchment masks for ice shelves

### Catchment boundary masks

In [ ]:
# Catchment boundary masks for Antarctica, taken from ice shelf definitions in MPAS-Dev/geometric-features
# Source: https://github.com/MPAS-Dev/geometric_features/tree/main/geometric_data
# These have been combined into one file with 133 defined regions (polygons and multipolygons), 
# readable via the Geopandas package

# Read geoJSON region feature file as GeoDataFrame
iceshelvesmask = gpd.read_file(inDirName + iceShelvesShape_filepath)

# Convert to south polar stereographic projection
#icems = iceshelvesmask.to_crs({'init': 'epsg:3031'}); # This has been deprecated
icems = iceshelvesmask.to_crs('epsg:3031');
crs = ccrs.SouthPolarStereo();

### Plot catchment boundaries

In [ ]:
# Plot boundaries

plt.figure(figsize=(5,5))
ax1 = plt.subplot(121,projection=ccrs.SouthPolarStereo())
ax1.gridlines(color='whitesmoke',zorder=4)
icems[34:133].plot(ax=ax1,color='antiquewhite', linewidth=0,zorder=1)
icems[34:133].boundary.plot(ax=ax1,color='r', linewidth=0.2,zorder=3)
#icems[34:133].boundary.plot(ax=ax1,linewidth=0.25,color='lightgray',zorder=4)
#ax1.coastlines(resolution='10m', zorder=6,linewidth=0.75)
ax1.patch.set_facecolor(color='lightsteelblue')
#ax1.add_feature(cartopy.feature.LAND, color='ghostwhite')
ax1.add_feature(cartopy.feature.LAND, color='ghostwhite', zorder=2)
plt.title('Catchment Boundaries');

### Ocean forcing data

In [ ]:
# Original forcing data: raw MPAS-O output, mapped to the 10km resolution grid
# flux is freshwater flux
# ssh is sea surface height, used here as a proxy for ice draft / depth.

# ssh is used to plot the scatterplot of flux vs. draft for different ice shelves 
# and as input for the linear regression used in "dedrafting" the dataset

regridded_data = xr.open_dataset(inDirName+regriddedFluxSSH_filepath)
flux = regridded_data.timeMonthly_avg_landIceFreshwaterFlux
ssh = regridded_data.timeMonthly_avg_ssh

# Pre-processed data: detrended, deseasonalized, dedrafted
flux_clean = xr.open_dataset(inDirName+interim_data_folder+'flux_clean')
flux_clean = flux_clean.timeMonthly_avg_landIceFreshwaterFlux

# Dedrafted flux
flux_dedrafted = xr.open_dataset(inDirName+
                                 interim_data_folder+
                                 flux_dedrafted_iceshelves_data_path+'iceshelves_dedrafted_total.nc')
flux_dedrafted = flux_dedrafted.timeMonthly_avg_landIceFreshwaterFlux

# Flux datapoints extracted for individual ice shelves, used for the scatter plots
catchments_scatter = np.load(inDirName+interim_data_folder+"catchments_scatter.npy")
catchments_scatter_xr = xr.DataArray(catchments_scatter,dims={'basin','x','y'})

# Add reconstructed datasets - seed random

In [ ]:
flux_clean = xr.open_dataset(inDirName+interim_data_folder+'flux_clean')
flux_clean = flux_clean.timeMonthly_avg_landIceFreshwaterFlux

In [ ]:
%%time
flux_regress = flux.polyfit('time',deg=2,full=True)

In [ ]:
flux_dedrafted = flux - flux_regress.polyfit_residuals

In [ ]:
# Deseasonalize
# Remove climatologies to isolate anomalies, i.e., internal variability
flux_month = flux_dedrafted.groupby("time.month")
flux_clm = flux_month.mean("time") # Climatologies
flux_anm = flux_month - flux_clm # Deseasonalized anomalies

In [ ]:
flux_anm_ts = flux_anm.sum(['y','x'])

In [ ]:
flux_clean_ts = flux_clean.sum(['y','x'])

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(flux_anm_ts[300:])
plt.plot(flux_clean_ts)

### Convert Freshwater Flux [$kg. m^{-2} s^{-1}$] to Melt Rate [$m.yr^{-1}$]
Assumes density of freshwater $\rho_{fw}$ [$kg/m^3$] = 1000

In [ ]:
sec_per_year = 365*24*60*60
rho_fw = 1000

In [ ]:
catchments = np.array([1,27,28,61,70,83,94,71,22])

### Plot sample data for specific ice shelves (melt rate draft dependence)

In [ ]:
# Figure: Melt rate draft dependence for select catchments
# The following indices are taken from 'catchment_scatter', defined as
# Index of catchments in catchment_scatter = Index of catchments in icems - 33

# Sample ice shelves chosen:
# Amery = 1
# George VI = 27
# Getz = 28
# Pine Island = 61
# Ronne = 70
# Totten = 83
# Wilkins = 94
# Western Ross = 71
# Filchner = 22
# Thwaites = 81
# Eastern_Ross = 72

# Full list of ice shelves and indices can be obtained by printing icems

catchments = np.array([1,27,28,61,70,83,94,71,22])

#fig, ax = plt.subplots(1, 1, figsize=[15, 15],subplot_kw={'projection': ccrs.SouthPolarStereo()})
fig, axs = plt.subplots(3,3, figsize=[15, 15])
axs=axs.ravel()

for i in range(9):
    axs[i].scatter(catchments_scatter[catchments[i]],ssh[1],color='r',s=4)
    #axs[i].set_xlabel('Melt Flux (kg / m2 / s)')
    #axs[i].set_ylabel('Depth (m)')
    axs[i].set_ylim(-1000,0) # Specified for consistency across plots, some catchments are not as deep as others
    # Uncomment to set xlim values for easy comparison of forcing magnitudes (varies across ice shelves)
    axs[i].set_xlim(-0.5e-6,2e-5)
    axs[i].set_title(icems.name[33+catchments[i]])
    # axs[i].inset_axes([], projection=crs) does not seem to work, hence the workaround below:
    # Refer here: https://github.com/matplotlib/matplotlib/pull/22608 for future revisions
    axins = inset_axes(axs[i],width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='r',linewidth=0.4)
    axins.set_axis_off();

axs[0].set_ylabel('Depth (m)'); # Ice Draft?
axs[3].set_ylabel('Depth (m)');
axs[6].set_ylabel('Depth (m)');


axs[6].set_xlabel('Forcing Flux ($kg m^{-1}s^{-2}$)');
axs[7].set_xlabel('Forcing Flux ($kg m^{-1}s^{-2}$)');
axs[8].set_xlabel('Forcing Flux ($kg m^{-1}s^{-2}$)');

# TODO: Convert to m SLE units for flux

# Data Pre-Processing

For time series plots and comparisons, the sum of flux across the spatial domain (or specific ice shelves) is used as a proxy when required. The mean can also be used here, without any loss of information, as the idea is only to compare trendlines.
#### TODO: Verify if this is indeed true

However, this might not be possible once the data is normalized, as the magnitude of data is removed at that stage.

In [ ]:
flux_ts = flux.sum(['x','y'])
flux_dedrafted_ts = flux_dedrafted.sum(['x','y'])

# Processed data time series
flux_clean_ts = flux_clean.sum(['x','y'])

In [ ]:
# Remove temporal mean
# Is this necessary?
flux_demeaned = flux - flux.mean('time')
flux_demeaned_ts = flux_demeaned.sum(['x','y'])

## Remove seasonality

In [ ]:
# Deseasonalize
# Remove climatologies to isolate anomalies, i.e., internal variability
flux_month = flux_dedrafted.groupby("time.month")
flux_clm = flux_month.mean("time") # Climatologies
flux_anm = flux_month - flux_clm # Deseasonalized anomalies

In [ ]:
flux_clm_ts = flux_clm.sum(['x','y'])
flux_anm_ts = flux_anm.sum(['x','y'])

In [ ]:
plt.figure(figsize=(25,6))
plt.plot(flux_ts[300:])
plt.plot(flux_dedrafted_ts[300:])
plt.plot(flux_dedrafted_ts - flux_clean_ts)
#plt.plot(flux_clean_ts)
plt.plot(flux_anm_ts[300:])

In [ ]:
# Sanity check, as "flux_clean.nc" was created earlier. The difference here is purely numerical.

plt.figure(figsize=(25,6))
plt.plot(flux_clean_ts - flux_anm_ts[300:])

# EOF/PCA Decomposition

## Filter data

We are primarily interested in interannual and interdecadal variability. Annual and sub-annual variability is filtered out via a low pass filter.

In [ ]:
# Define Butterworth filter parameters 
fs=30/24/3600 # 1 month in Hz (sampling frequency)
nyquist = fs / 2 # 0.5 times the sampling frequency
cutoff=0.15 # fraction of nyquist frequency, chosen to define the cutoff frequency
print('cutoff= ',1/cutoff*nyquist*24*3600) # cutoff=  __  days
b, a = signal.butter(1, cutoff, btype='lowpass') #low pass filter, 5 is chosen as order of filter

In [ ]:
1

In [ ]:
%%time
y = flux_clean.shape[1]
x = flux_clean.shape[2]

flux_clean_filtered = np.empty((flux_clean.shape[0],y,x))

for i in range(y):
    for j in range(x):
        fl_filt = signal.filtfilt(b, a, flux_clean[:,i,j])
        fl_filt = np.array(fl_filt)
        fl_filt = fl_filt.transpose()
        flux_clean_filtered[:,i,j] = fl_filt
        print("filtered at {},{}".format(i,j))

In [ ]:
flux_clean_filtered = xr.DataArray(flux_clean_filtered,
                                   dims=flux_clean_normalized.dims,
                                   coords=flux_clean_normalized.coords,
                                   attrs=flux_clean_normalized.attrs)

flux_clean_filtered.to_netcdf(inDirName+interim_data_folder+
                              cise_file_path+std_file_path+temp_filtered_path+"flux_clean_filtered_Order1.nc")

flux_clean_filtered = xr.open_dataset(inDirName+interim_data_folder+
                                      cise_file_path+std_file_path+temp_filtered_path+"flux_clean_filtered_Order1.nc")

flux_clean_filtered = flux_clean_filtered.__xarray_dataarray_variable__

In [ ]:
flux_clean_filtered_ts = time_series(flux_clean_filtered)

In [ ]:
plt.psd(flux_clean_filtered_ts);

### Data Normalization

In [ ]:
flux_clean_tmean = flux_clean.mean('time')
flux_clean_tstd = flux_clean.std('time')

flux_clean_demeaned = flux_clean - flux_clean_tmean
# Why does this introduce a non-zero change? (Artefact of flux variable dimensions?)
flux_clean_normalized = flux_clean_demeaned/flux_clean_tstd

In [ ]:
flux_clean_filtered_tmean = flux_clean_filtered.mean('time')
flux_clean_filtered_tstd = flux_clean_filtered.std('time')

flux_clean_filtered_demeaned = flux_clean_filtered - flux_clean_filtered_tmean
# Why does this introduce a non-zero change? (Artefact of flux variable dimensions?)
flux_clean_normalized = flux_clean_filtered_demeaned/flux_clean_filtered_tstd

Time series of the normalized data is calculated below only for illustrative purposes. It would not make sense to compare the normalized time series to other time series (which use the spatial sum or mean).

In [ ]:
flux_clean_normalized_ts = flux_clean_normalized.sum(['x','y'])

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(flux_clean_normalized_ts,color = '#44546A', linewidth=2)
plt.title('Anomalies / Internal Variability Component (Normalized)');
plt.ylabel('Standard Deviation Units');
plt.xlabel('Time (months)');

In [ ]:
plt.figure(figsize=(25,8))
filtered_flux_ts.plot()
flux_clean_normalized_ts.plot()

## Spatial Decomposition: EOF Analysis

EOF decomposition is done using a modified version of the `xeofs` package.
Refer [nicrie/xeofs](https://github.com/nicrie/xeofs) on Github for documentation.

#### TODO: Upload modification of `xeofs` online, or rewrite Xarray wrapper for the scikit-learn PCA to encompass full workflow

In [ ]:
%%time
model = EOF(flux_clean_normalized) # change accordingly if using original data, or normalized data
#model = EOF(flux_clean) # change accordingly if using original data, or normalized data
model.solve()
eofs = model.eofs()
pcs = model.pcs()
nmodes = model.n_modes
varexpl = model.explained_variance_ratio()
pcs_eig = model.pcs(1)
pcs_sng = model.pcs(2)

In [ ]:
%%time
# T-mode
model = EOF(flux_clean_normalized, dim=['y', 'x'])
model.solve()
pcs = model.eofs()
eofs = model.pcs()
nmodes = model.n_modes
varexpl = model.explained_variance_ratio()

### Plot relative power of first m modes from the PCA decomposition

In [ ]:
# Figure: Relative power of m EOFs
m = 300
sns.set_theme(style="white")
plt.figure(figsize=(15,8))
plt.bar(range(m),varexpl[:m]*100, 
        color='black')
plt.ylabel('Relative Power (\%)')
plt.xlabel('EOF Mode')
plt.title('Total Relative Power of first {} EOFs: {:.2f}\%'.format(m, varexpl[:m].sum().values*100));

### Plot dominant modes of variability: EOF spatial modes and PC series
The plots show dominant spatial and temporal modes of variability for the normalized dataset. It is difficult to interpret these physically.
#### TODO: Define vmax, vmin params for eofs.plot() if plotting for original cleaned dataset (un-normalized data)

In [ ]:
nmodes_plot = 3 # Number of modes to plot

sns.set_theme(style="white")
fig=plt.figure(figsize=(15,8))
gs = GridSpec(nmodes_plot,2,width_ratios=[1.5,2.5])
ax0 = [fig.add_subplot(gs[i, 0],projection=crs) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0, a1) in enumerate(zip(ax0, ax1)):
    eofs.sel(mode=i+1).plot(ax=a0,
                            cmap='cmo.balance',
                            add_colorbar=True,
                            cbar_kwargs={'orientation': 'vertical',
                                         'label': 'Standard Deviation Units'})
    a0.coastlines(resolution='10m',linewidth=0.25,color='0.8')
    a1.plot(pcs.sel(mode=i+1),linewidth=1,color='dimgray')
    a1.set_xlabel('')

a1.set_xlabel('Time (months)');
#cb = plt.colorbar(eofs.sel(mode=1), ax=a0, orientation="vertical", pad=0.15)


"""
# Figure: Dominant EOF modes
sns.set_theme(style="white")
fg = eofs[:,:,0:2].plot(aspect=2,
                        size=8,
                        col="mode", 
                        col_wrap=2,
                        cmap='cmo.balance', # Uses cmocean colorbar - should be installed
                        cbar_kwargs={'orientation': 'vertical',
                                    'label': 'Standard Deviation Units'},
                        subplot_kws=dict(projection=ccrs.SouthPolarStereo()))

[ax.coastlines() for ax in fg.axs.flatten()];
""";

## Fourier Phase Randomization

Now, the projection co-efficients from the previous step are phase-randomized to generate new realizations of the data. In this step, we sample phases for each FFT(PC) from (0,$2\pi$), and iterate to generate n realizations.

#### 0. No PC filtering (filter done on input dataset before EOF decomposiition

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_fourier = np.fft.rfft(fl)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

In [ ]:
plt.magnitude_spectrum(pcs[:,50]);
plt.magnitude_spectrum(new_fl_nothing[0,:,50]);

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(pcs[:,0:2]);
#plt.plot(pcs[:,2:4]);

In [ ]:
m = 10
t = len(pcs[:,0]) - m
E = 0

E_t = np.array([])
E = 0

for t in range(t):
    for i in range(m):
        E = E + (pcs[i,0] - pcs[t+i,0])**2
    print(E_t)
    E_t = np.append(E_t, E) 

In [ ]:
plt.plot(E_t[1:])

In [ ]:
### Plot CDFs
cdfx = np.sort(fl_fourier)
cdfy = np.arange(1500) / float(1500)

cdfxorig = np.sort(random_phases)
# plotting  
plt.title('CDF using sorting the data')
#plt.plot(cdfx, marker='.')
plt.plot(cdfxorig, marker='.')

### IAAFT Surrogate modeling (alternative to normal FFT)

In [ ]:
xs = np.empty((1500,2,1500))
for i in range(1500):
    x = iaaft.surrogates(x=pcs[:,i], ns=2, verbose=True)
    xs[i] = x

### Define Fourier filter
This section makes use of filters (Butterworth filter in particular) to cutoff high frequency variability components, which might be spurious artefacts of the high dimensionality dataset, and can lead to noise in the generated outputs.

In [ ]:
# Define Butterworth filter parameters 
fs=30/24/3600 # 1 month in Hz (sampling frequency)
nyquist = fs / 2 # 0.5 times the sampling frequency
cutoff=0.15 # fraction of nyquist frequency, chosen to define the cutoff frequency
print('cutoff= ',1/cutoff*nyquist*24*3600) # cutoff=  __  days
bl, al = signal.butter(4, cutoff, btype='lowpass') #low pass filter, 1 is chosen as order of filter
bh, ah = signal.butter(4, cutoff, btype='highpass') #high pass filter, 1 is chosen as order of filter

### Fourier functions

In [ ]:
def butter_lowpass(cutoff, fs, order=1):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=1):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_highpass(cutoff, fs, order=3):
    return butter(order, cutoff, fs=fs, btype='high', analog=False)

def butter_highpass_filter(data, cutoff, fs, order=3):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
order = 1
fs = 30/24/3600     # 30/24/3600  # sample rate, Hz
cutoff = 1/24

### Randomization over filtered PCs

#### 1. Filter PCs before randomization

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_filt = signal.filtfilt(bl, al, fl)
        fl_filt=np.array(fl_filt)
        fl_filt=fl_filt.transpose()
        fl_fourier = np.fft.rfft(fl_filt)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl_prerandom = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_filt = signal.filtfilt(bl, al, fl)
        fl_filt=np.array(fl_filt)
        fl_filt=fl_filt.transpose()
        fl_fourier = np.fft.rfft(fl_filt)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl_prerandom[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

In [ ]:
### Plot CDFs
cdfx = np.sort(fl_fourier_new)
cdfy = np.arange(1500) / float(1500)

cdfxorig = np.sort(fl_fourier)
# plotting  
plt.title('CDF using sorting the data')
plt.plot(cdfx, marker='.')
plt.plot(cdfxorig, marker='.')

#### 1.5 Combine (1) and (2): Low-Pass filter before randomization, High-Pass filter after randomization

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_filt = signal.filtfilt(bl, al, fl)
        fl_filt=np.array(fl_filt)
        fl_filt=fl_filt.transpose()
        fl_fourier = np.fft.rfft(fl_filt)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl_filt = np.fft.irfft(fl_fourier_new)
        new_fl_filt = signal.filtfilt(bh, ah, new_fl_filt)
        new_fl_filt = np.array(new_fl_filt)
        new_fl_filt = new_fl_filt.transpose()
        new_fl[i,:,m] = new_fl_filt + np.fft.irfft(fl_fourier_new)
        # new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

#### 1.5 [b] Combine (1) and (2): Low-Pass filter before randomization, High-Pass filter after randomization, USE FUNCTIONS

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_filt = butter_lowpass_filter(fl,cutoff,fs,order)
        #fl_filt = signal.filtfilt(bl, al, fl)
        #fl_filt=np.array(fl_filt)
        #fl_filt=fl_filt.transpose()
        fl_fourier = np.fft.rfft(fl_filt)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl_filt = np.fft.irfft(fl_fourier_new)
        new_fl_filt = butter_highpass_filter(new_fl_filt,cutoff,fs,order)
        new_fl[i,:,m] = new_fl_filt + np.fft.irfft(fl_fourier_new)
        # new_fl[i,:,m] = np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

#### 2. Filter PCs after randomization (before generation)

In [ ]:
%%time
##############################
# FOURIER PHASE RANDOMIZATION 
##############################

# Define number of random Fourier realizations
n_realizations = 2
t_length = pcs.shape[0]

# Define random number generator 
#rng = np.random.default_rng(2021)
#random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j) in line 26

# xeofs_pcs[:,i] when using PCA outputs
new_fl = np.empty((n_realizations,pcs.shape[0],pcs.shape[1]))

# Time limits for plotting
t1 = 0
tf = int(t_length/2)

for i in range(n_realizations):
    for m in range(nmodes):
        fl = pcs[:,m] # fluxpcs[:,i] when using PCA outputs
        fl_fourier = np.fft.rfft(fl)
        #random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        #random_phases = np.exp(np.random.default_rng(2023).uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        random_phases = np.exp(np.random.uniform(0,2*np.pi,int(len(fl)/2+1))*1.0j)
        fl_fourier_new = fl_fourier*random_phases
        new_fl_filt = np.fft.irfft(fl_fourier_new)
        new_fl_filt = signal.filtfilt(b, a, new_fl_filt)
        new_fl_filt = np.array(new_fl_filt)
        new_fl_filt = new_fl_filt.transpose()
        new_fl[i,:,m] = new_fl_filt #np.fft.irfft(fl_fourier_new)
    print('calculated ifft for realization {}, all modes'.format(i))

### Plot PCs and their PSDs

PCs are generated using 1 of the three methods above

In [ ]:
nmodes_plot = 4 # Number of modes to plot
nrealizations_to_plot = 2 # to be lesser than absolute total number, defined in the Fourier randomization step

sns.set_theme(style="white")
fig=plt.figure(figsize=(25,20))

gs = GridSpec(nmodes_plot, 2, width_ratios=[4, 2])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0,a1) in enumerate(zip(ax0,ax1)):
    for n_realization in range(0,nrealizations_to_plot):
        a0.plot(new_fl[n_realization,:,i],color='b', linewidth=0.5)
        a1.psd(new_fl[n_realization,:,i],color='b', linewidth=0.5)
    a0.plot(new_fl[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a1.psd(new_fl[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a0.plot(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a1.psd(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a0.set_title('PC for EOF mode {}'.format(i+1))
    a1.set_title('PSD for PC mode {}'.format(i+1))
    a1.set_xlabel('')

a0.set_xlabel('Time (months)')
a1.set_xlabel('Frequency')
plt.legend();

In [ ]:
### Plot CDFs
cdfx = np.sort(new_fl[0,:,0])
cdfxnewfilt = np.sort(new_fl_prerandom[0,:,0])
cdfy = np.arange(1500) / float(1500)
cdfxorig = np.sort(pcs[:,0])
# plotting  
plt.title('CDF using sorting the data')
plt.plot(cdfx, marker='.')
plt.plot(cdfxorig, marker='.')
plt.plot(cdfxnewfilt, marker='.')

In [ ]:
import matplotlib as mpl

ncolors=5
nstart = 0
cmap1 = mpl.colormaps['Greys'](np.linspace(0,1,ncolors))
cmap2 = mpl.colormaps['Blues'](np.linspace(0,1,ncolors))
cmap3 = mpl.colormaps['Oranges'](np.linspace(0,1,ncolors))
cmap4 = mpl.colormaps['Greens'](np.linspace(0,1,ncolors))
plt.figure(figsize=(15,8))

plt.psd(pcs[:,nstart],noverlap=8,NFFT=256,color=cmap1[ncolors-1], label='Input data (Actual)');
plt.psd(new_fl_nothing[0,:,nstart],noverlap=8,NFFT=256,color=cmap2[ncolors-1], label='Randomized PCs (No filtering)');
plt.psd(new_fl_prerandom[0,:,nstart],noverlap=8,NFFT=256,color=cmap2[ncolors-1], label='Filter PCs before Randomization');
plt.psd(new_fl[0,:,nstart],noverlap=8,NFFT=256,color=cmap3[ncolors-1], label='Filter Randomized PCs before Generation');

for i in range(nstart,nstart+ncolors):
    plt.psd(pcs[:,i],noverlap=8,NFFT=256,color=cmap1[i-nstart]);
    plt.psd(new_fl_prerandom[0,:,i],noverlap=8,NFFT=256,color=cmap2[i-nstart]);
    plt.psd(new_fl_nothing[0,:,i],noverlap=8,NFFT=256,color=cmap4[i-nstart]);
    plt.psd(new_fl[0,:,i],noverlap=8,NFFT=256,color=cmap3[i-nstart]);
plt.legend()

In [ ]:
nmodes_plot = 4 # Number of modes to plot
nrealizations_to_plot = 2 # to be lesser than absolute total number, defined in the Fourier randomization step

sns.set_theme(style="white")
fig=plt.figure(figsize=(25,20))

gs = GridSpec(nmodes_plot, 2, width_ratios=[4, 2])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0,a1) in enumerate(zip(ax0,ax1)):
    for n_realization in range(0,nrealizations_to_plot):
        a0.plot(new_fl[n_realization,:,i],color='b', linewidth=0.5)
        a1.psd(new_fl[n_realization,:,i],color='b', linewidth=0.5)
    a0.plot(new_fl[0,:,i]+new_fl_high[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a1.psd(new_fl[0,:,i]+new_fl_high[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a0.plot(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a1.psd(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a0.set_title('PC for EOF mode {}'.format(i+1))
    a1.set_title('PSD for PC mode {}'.format(i+1))
    a1.set_xlabel('')

a0.set_xlabel('Time (months)')
a1.set_xlabel('Frequency')
plt.legend();

## Statistical Generation

This uses a custom function presently committed to the `xeofs` package. There is the possibility that this might break should `xeofs` be upgraded.

*Ref: For commit diff details, see [mshiv/xeofs-rand](https://github.com/nicrie/xeofs/compare/main...mshiv:xeofs-rand:main).*
#### TODO: Rewrite generator function using PCA as a standalone package and remove `xeofs` dependency.

In [ ]:
#modified to not return f - in calculation of RMSE, only Px required
def psd_calc_grid(data,y,x):
    f, Px = scipy.signal.welch(data[:,y,x])
    return Px

def time_series(clipped_data):
    clipped_ts = clipped_data.sum(['y','x'])
    return clipped_ts

# Reconstruct flux dataset using phase randomized PCs
# This section is to be called iteratively for ensemble runs with multiple realizations
# This method also takes 'modes' as a parameter: 
# which is used to reconstruct dataset with different number of selected modes

def generate_data(n_realization,mode,mode_skip):
    flux_reconstr = model.reconstruct_randomized_X(new_fl[n_realization],slice(1,mode,mode_skip))
    #flux_reconstr = flux_reconstr.dropna('time',how='all')
    #flux_reconstr = flux_reconstr.dropna('y',how='all')
    #flux_reconstr = flux_reconstr.dropna('x',how='all')
    #flux_reconstr = flux_reconstr.drop("month")
    return flux_reconstr

def generate_noise(n_realization,mode,mode_skip):
    flux_reconstr_noise = model.reconstruct_randomized_X(new_fl_high[n_realization],slice(1,mode,mode_skip))
    #flux_reconstr = flux_reconstr.dropna('time',how='all')
    #flux_reconstr = flux_reconstr.dropna('y',how='all')
    #flux_reconstr = flux_reconstr.dropna('x',how='all')
    #flux_reconstr = flux_reconstr.drop("month")
    return flux_reconstr_noise


def clip_data(total_data, basin):
    clipped_data = total_data.rio.clip(icems.loc[[basin],'geometry'].apply(mapping))
    #clipped_data = clipped_data.dropna('time',how='all')
    #clipped_data = clipped_data.dropna('y',how='all')
    #clipped_data = clipped_data.dropna('x',how='all')
    clipped_data = clipped_data.drop("month")
    return clipped_data

In [ ]:
# Generate dataset realizations
# Note standardized input data to the initial PCA 
for i in range(2):
    flux_reconstr = generate_data(i, 1500, 1)
    #flux_reconstr_noise = generate_noise(i, 1500, 1)
    flux_reconstr = (flux_reconstr*flux_clean_tstd)+flux_clean_tmean
    #flux_reconstr_noise = (flux_reconstr_noise*flux_clean_tstd)+flux_clean_tmean
    #flux_reconstr = flux_reconstr + flux_reconstr_noise
    flux_reconstr.to_netcdf(inDirName+interim_data_folder+
                            cise_file_path+std_file_path+temp_filtered_path+"REC_{}.nc".format(i))
    del flux_reconstr
    gc.collect()
    print('reconstructed realization # {}'.format(i))

In [ ]:
correlation = xr.corr(flux_clean, flux_reconstr0, dim='time')

In [ ]:
flux_clean

### Reconstruct with IAAFT surrogates

In [ ]:
def generate_iaaft_data(n_realization,mode,mode_skip):
    flux_reconstr = model.reconstruct_randomized_X(xsnew[n_realization],slice(1,mode,mode_skip))
    #flux_reconstr = flux_reconstr.dropna('time',how='all')
    #flux_reconstr = flux_reconstr.dropna('y',how='all')
    #flux_reconstr = flux_reconstr.dropna('x',how='all')
    #flux_reconstr = flux_reconstr.drop("month")
    return flux_reconstr

In [ ]:
xsnew = np.moveaxis(xs,0,-1)

In [ ]:
# Generate dataset realizations
# Note standardized input data to the initial PCA 
for i in range(2):
    flux_reconstr = generate_iaaft_data(i, 1500, 1)
    #flux_reconstr_noise = generate_noise(i, 1500, 1)
    flux_reconstr = (flux_reconstr*flux_clean_tstd)+flux_clean_tmean
    #flux_reconstr_noise = (flux_reconstr_noise*flux_clean_tstd)+flux_clean_tmean
    #flux_reconstr = flux_reconstr + flux_reconstr_noise
    flux_reconstr.to_netcdf(inDirName+interim_data_folder+
                            cise_file_path+std_file_path+"iaaft/REC_{}.nc".format(i))
    del flux_reconstr
    gc.collect()
    print('reconstructed realization # {}'.format(i))

In [ ]:
# change file name for other realizations
flux_reconstr0 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'iaaft/REC_0.nc')
flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__

flux_reconstr1 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+'iaaft/REC_1.nc')
flux_reconstr1 = flux_reconstr1.__xarray_dataarray_variable__

In [ ]:
modes_used = [200, 500, 1500]
catchments = np.array([1,27,28,61,70,83])
#catchments = np.array([1,27,28,61])
basins = catchments+33
n_basins = len(basins)
color_orig = 'k'
color_gen = 'dimgray'

sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5*n_basins))

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for i,(a0,a1) in enumerate(zip(ax0,ax1)):
    orig = clip_data(flux_clean, basins[i])
    rec0 = clip_data(flux_reconstr0, basins[i])
    rec1 = clip_data(flux_reconstr1, basins[i])
    #rec2 = clip_data(flux_reconstr2, basins[i])
    orig = orig.rename('orig')
    rec0 = rec0.rename('rec0')
    rec1 = rec1.rename('rec1')
    #rec2 = rec2.rename('rec2')
    combined = xr.merge([orig,rec0,rec1])
    combined_ts = time_series(combined)
    a0.plot(combined_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a0.plot(combined_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a0.plot(combined_ts.rec1,color=color_gen, linewidth=0.25)
    #a0.plot(combined_ts.rec2,color=color_gen, linewidth=0.25)
    a1.psd(combined_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a1.psd(combined_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a1.psd(combined_ts.rec1,color=color_gen, linewidth=0.25)
    #a1.psd(combined_ts.rec2,color=color_gen, linewidth=0.25)
    a0.set_title('Time Series: {}'.format(icems.name[basins[i]]))
    a1.set_title('PSD: {}'.format(icems.name[basins[i]]))
    a0.set_xlabel('')
    a1.set_xlabel('')
    axins = inset_axes(a0, width="30%", height="30%", loc="upper right", 
                       axes_class=cartopy.mpl.geoaxes.GeoAxes, 
                       axes_kwargs=dict(projection=ccrs.SouthPolarStereo()))
    icems[33:133].plot(ax=axins,linewidth=0.3,color='lightsteelblue')
    axins.coastlines(resolution='10m',linewidth=0.25)
    icems.loc[[33+catchments[i]],'geometry'].plot(ax=axins,color='k',linewidth=0.4)
    del orig, rec0, rec1, combined
a1.legend();
a0.set_xlabel('Time (months)');
a1.set_xlabel('Frequency');
#plt.savefig(inDirName+processed_data_folder+pca_3+'generator-compare_{}-modes'.format(mode_number))
gc.collect()

## Helper Functions

Functions defined for the generation and follow on calculations

## Input generated dataset files

Versions of these files from different experimental runs are currently housed in the project repo at ```/data/interim```. Note that the ```/data``` directory is ignored for remote commits, and is currently available only on local.
Location can be changed above when saving the file to NETCDF in the Generation step.

In [ ]:
# change file name for other realizations
flux_reconstr0 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+temp_filtered_path+'REC_0.nc')
flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__

flux_reconstr1 = xr.open_dataset(inDirName+interim_data_folder+
                                 cise_file_path+std_file_path+temp_filtered_path+'REC_1.nc')
flux_reconstr1 = flux_reconstr1.__xarray_dataarray_variable__

In [ ]:
# Specify catchments to clip data and plot for comparisons
#Old reconstructions: 
# basins = np.array((34,55,114,103,104,105))
# From previous section on chosen catchments (Scatter plot):
#catchments = np.array([1,27,28,61,70,83,94,71,22])
catchments = np.array([1,27,28,61])
basins = catchments+33

# Sample ice shelves chosen:
# Amery = 1 ~ 34
# George VI = 27 ~ 
# Getz = 28
# Pine Island = 61
# Ronne = 70
# Totten = 83
# Wilkins = 94
# Western Ross = 71
# Filchner = 22
# Thwaites = 81
# Eastern_Ross = 72

Create subsets of data (original and reconstructed realizations) for individual catchments. The data variables in each are named so that they may be easily input using the `xarray.open_mfdataset` method. Note however that `CDO` terminal commands are better suited for this if the number of realizations is significantly high (beyond 30?)

#### Clip data to each basin

In [ ]:
for basin in basins:
    orig = clip_data(flux_clean, basin)
    #orig = clip_data(flux_clean_filtered, basin)
    rec0 = clip_data(flux_reconstr0, basin)
    rec1 = clip_data(flux_reconstr1, basin)
    orig.rename('orig'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+temp_filtered_path+"orig_{}_filt.nc".format(basin))
    rec0.rename('rec0'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+temp_filtered_path+"rec0_{}_filt.nc".format(basin))
    rec1.rename('rec1'.format(basin)).to_netcdf(inDirName+interim_data_folder+
                                                   cise_file_path+std_file_path+temp_filtered_path+"rec1_{}_filt.nc".format(basin))
    del orig, rec0, rec1
    gc.collect()

#### Combine realizations and original data as separate data variables in `NETCDF` files.

In [ ]:
for basin in basins:
    basin_combined_datasets = xr.open_mfdataset(inDirName+interim_data_folder+
                                                cise_file_path+std_file_path+
                                                temp_filtered_path+"*{}_filt.nc".format(basin),
                                                combine="nested")
    basin_combined_datasets.to_netcdf(inDirName+interim_data_folder+
                                      cise_file_path+std_file_path+
                                      temp_filtered_path+"combined_{}.nc".format(basin))
    del basin_combined_datasets
    gc.collect()

#### Plot figure (time series and PSD comparisons for each basin)

In [ ]:
n_basins = len(basins)

color_orig = 'k'
color_gen = 'dimgray'

sns.set_theme(style="white")
fig=plt.figure(figsize=(18,5*n_basins))

gs = GridSpec(n_basins, 2, width_ratios=[3, 1])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(n_basins)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(n_basins)]

for i,(a0,a1) in enumerate(zip(ax0,ax1)):
    combined_ds = xr.open_dataset(inDirName+interim_data_folder+
                              cise_file_path+std_file_path+temp_filtered_path+"combined_{}.nc".format(basins[i]))
    combined_ds_ts = time_series(combined_ds)
    a0.plot(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a0.plot(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a0.plot(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a1.psd(combined_ds_ts.orig,color=color_orig, linewidth=1.5,label='Original')
    a1.psd(combined_ds_ts.rec0,color=color_gen, linewidth=0.25,label='Randomized')
    a1.psd(combined_ds_ts.rec1,color=color_gen, linewidth=0.25)
    a0.set_title('Time Series: {}'.format(icems.name[basins[i]]))
    a1.set_title('PSD: {}'.format(icems.name[basins[i]]))
    a0.set_xlabel('')
    a1.set_xlabel('')
    #axins.set_axis_off();
#a0.legend();
a1.legend();
a0.set_xlabel('Time (months)');
a1.set_xlabel('Frequency');
#gs.tight_layout(fig)

In [ ]:
%%time
model = EOF(flux_reconstr0) # change accordingly if using original data, or normalized data
#model = EOF(flux_clean) # change accordingly if using original data, or normalized data
model.solve()
eofsnew = model.eofs()
pcsnew = model.pcs()

In [ ]:
eofsnew[:,:,100].plot()

### Plot comparisons of full AIS time series

#### Input full dataset (if not already done) and sum over entire domain to obtain time series.

In [ ]:
# Full AIS data comparisons
flux_clean_ts = time_series(flux_clean)
flux_reconstr0_ts = time_series(flux_reconstr0)
flux_reconstr1_ts = time_series(flux_reconstr1)

#### Plot figure

#### TODO: Add formatting and animate spatial variability fields for 2 generated realizations and original data. 
This will be added as a supplement.

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(25,8))

plt.plot(flux_clean_ts, color='k', linewidth=1.5,label='Original')
#plt.plot(filtered_flux_ts, color='k', linewidth=1.5,label='Original')
plt.plot(flux_reconstr0_ts, color='b', linewidth=0.25,label='Randomized')
#plt.plot(flux_reconstr1_ts, color='b', linewidth=0.25)

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(25,8))

plt.plot(flux_clean_ts*sec_per_year/rho_fw, color='k', linewidth=1.5,label='Original')
plt.plot(flux_reconstr0_ts*sec_per_year/rho_fw, color='b', linewidth=0.25,label='Randomized')
plt.plot(flux_reconstr1_ts*sec_per_year/rho_fw, color='b', linewidth=0.25)

In [ ]:
nmodes_plot = 4 # Number of modes to plot
nrealizations_to_plot = 2 # to be lesser than absolute total number, defined in the Fourier randomization step

sns.set_theme(style="white")
fig=plt.figure(figsize=(25,20))

gs = GridSpec(nmodes_plot, 2, width_ratios=[4, 2])
ax0 = [fig.add_subplot(gs[i, 0]) for i in range(nmodes_plot)]
ax1 = [fig.add_subplot(gs[i, 1]) for i in range(nmodes_plot)]

for i, (a0,a1) in enumerate(zip(ax0,ax1)):
    for n_realization in range(0,nrealizations_to_plot):
        a0.plot(xsnew[n_realization,:,i],color='b', linewidth=0.5)
        a1.psd(xsnew[n_realization,:,i],color='b', linewidth=0.5)
    a0.plot(xsnew[0,:,i],color='b', linewidth=0.25,label='Randomized')
    a1.psd(xsnew[0,:,i],color='b', noverlap=120,NFFT=256,linewidth=0.25,label='Randomized')
    a0.plot(pcs[:,i],color='k', linewidth=2.5,label='Original')
    a1.psd(pcs[:,i],color='k', noverlap=120,NFFT=256,linewidth=2.5,label='Original')
    a0.set_title('PC for EOF mode {}'.format(i+1))
    a1.set_title('PSD for PC mode {}'.format(i+1))
    a1.set_xlabel('')

a0.set_xlabel('Time (months)')
a1.set_xlabel('Frequency')
plt.legend();

In [ ]:
sns.set_theme(style="white")
fig=plt.figure(figsize=(15,8))

plt.subplot(211)
plt.plot(flux_clean_ts, color='k', linewidth=1.5,label='Original');
plt.plot(flux_reconstr0_ts, color='b', linewidth=0.25,label='Randomized');
plt.plot(flux_reconstr1_ts, color='b', linewidth=0.25);

plt.subplot(212)
plt.psd(flux_clean_ts, color='k', linewidth=1.5,label='Original');
plt.psd(flux_reconstr0_ts, color='b', linewidth=0.25,label='Randomized');
plt.psd(flux_reconstr1_ts, color='b', linewidth=0.25);

## PSD calculations for AIS domain

In [ ]:
from scipy import signal
freqs0, psd0 = signal.welch(flux_clean_ts)
freqs1, psd1 = signal.welch(flux_reconstr0_ts)
freqs2, psd2 = signal.welch(flux_reconstr1_ts)

In [ ]:
from scipy import signal
freqs0, psd0 = signal.welch(combined_ds_ts.orig)
freqs1, psd1 = signal.welch(combined_ds_ts.rec0)
freqs2, psd2 = signal.welch(combined_ds_ts.rec1)

## PSD plot for AIS domain

This uses a normalized (semilog?) power in the y-axis, as opposed to [dB/Hz] shown earlier.

#### TODO: Check all PSDs for different catchments

In [ ]:
plt.figure(figsize=(10, 4))
plt.semilogx(freqs0, psd0,color='k', linewidth=1.5,label='Original')
plt.semilogx(freqs1, psd1,color='b', linewidth=1.25,label='Randomized')
plt.semilogx(freqs2, psd2,color='b', linewidth=1.25)
plt.ylabel('Power')
plt.xlabel('Frequency ($month^{-1}$)')

### Spectral Filtering on dataset

### TODO: 
#### 3 tests:
    - [ ] Filter on original data input - easiest?
    - [ ] Filter on projection co-efficients before randomization
    - [ ] Filter on generated projection co-efficients after randomization, before generation
    - [ ] Filter after generation - NO

In [ ]:
# Define Butterworth filter parameters 
fs=30/24/3600 # 1 month in Hz (sampling frequency)
nyquist = fs / 2 # 0.5 times the sampling frequency
cutoff=0.15 # fraction of nyquist frequency, chosen to define the cutoff frequency
print('cutoff= ',1/cutoff*nyquist*24*3600) # cutoff=  __  days
b, a = signal.butter(1, cutoff, btype='lowpass') #low pass filter, 5 is chosen as order of filter

In [ ]:
mode_number = 1500
flux_reconstr0 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_0.nc'.format(mode_number))
flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__

In [ ]:
mode_number = 1500
flux_reconstr0 = xr.open_dataset(inDirName+processed_data_folder+pca_3+'REC_{}-modes_0.nc'.format(mode_number))
flux_reconstr0 = flux_reconstr0.__xarray_dataarray_variable__

In [ ]:
fl_filt = signal.filtfilt(b, a, flux_reconstr0, axis=0) # axis=time axis (0) 
fl_filt = xr.DataArray(fl_filt, dims = flux_reconstr0.dims, coords = flux_reconstr0.coords, attrs = flux_reconstr0.attrs)

In [ ]:
flux_reconstr0_ts = time_series(flux_reconstr0) # Filter Randomized PC before Generation

In [ ]:
flux_reconstr0_pre_random = xr.open_dataset(inDirName+interim_data_folder+
                                            cise_file_path+std_file_path+temp_filtered_path+'REC_0.nc')
flux_reconstr0_pre_random = flux_reconstr0_pre_random.__xarray_dataarray_variable__
flux_reconstr0_pre_random_ts = time_series(flux_reconstr0_pre_random)

In [ ]:
flux_filtered_input_reconstruction = xr.open_dataset(inDirName+interim_data_folder+
                                                     cise_file_path+std_file_path+
                                                     temp_filtered_path+"1500_modes_24mon_cutoff/REC_0.nc")
flux_filtered_input_reconstruction = flux_filtered_input_reconstruction.__xarray_dataarray_variable__
flux_filtered_input_reconstruction_TS = time_series(flux_filtered_input_reconstruction)

In [ ]:
flux_filtered_reconstruction_noise = xr.open_dataset(inDirName+interim_data_folder+
                                                     cise_file_path+std_file_path+
                                                     temp_filtered_path+"1500_modes_24mon_cutoff/flux_clean_reconstr_noise_24mon.nc")

flux_filtered_input_reconstruction = flux_clean - flux_filtered_reconstruction_noise
flux_filtered_input_reconstruction_ts = time_series(flux_filtered_input_reconstruction)

In [ ]:
flux_filtered_input_reconstruction_ts = flux_filtered_input_reconstruction_ts.__xarray_dataarray_variable__

In [ ]:
flux_clean.plot()

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(flux_clean_ts, color='k', lw=2,label='Input data (Actual)');
plt.plot(flux_filtered_input_reconstruction_ts, color='gray',lw=0.5,label='Filter Generator Input')
plt.plot(flux_reconstr0_pre_random_ts,color='b', lw=0.5,label='Filter PCs before Randomization');
plt.plot(flux_reconstr0_ts, color='g',lw=0.5,label='Filter Randomized PCs before Generation');
plt.plot(fl_filt_ts, color='r',lw=0.5,label='Filter Generated Output');
plt.legend()

In [ ]:
plt.figure(figsize=(12,8))
plt.psd(flux_clean_ts, color='k', lw=2,label='Input data (Actual)');
#plt.psd(flux_filtered_input_reconstruction_ts, color='m',lw=0.5,label='Filter Generator Input')
plt.psd(flux_reconstr0_pre_random_ts,color='b', lw=1.5,label='Filter PCs before Randomization');
plt.psd(flux_reconstr0_ts, color='g',lw=0.5,label='Filter Randomized PCs before Generation');
plt.psd(fl_filt_ts, color='r',lw=0.5,label='Filter Generated Output');
plt.legend()

In [ ]:
flux_clean_ts = time_series(flux_clean)
fl_filt_ts = time_series(fl_filt)

In [ ]:
plt.figure(figsize=(25,8))
plt.plot(flux_clean_ts)
plt.plot(fl_filt_ts)

In [ ]:
plt.psd(flux_clean_ts)
plt.psd(fl_filt_ts)

In [ ]:
%%time
y = flux_clean.shape[1]
x = flux_clean.shape[2]

flux_gen_filtered = np.empty((flux_clean.shape[0],y,x))

for i in range(y):
    for j in range(x):
        fl_filt = signal.filtfilt(b, a, flux_reconstr0[:,i,j])
        fl_filt = np.array(fl_filt)
        fl_filt = fl_filt.transpose()
        flux_gen_filtered[:,i,j] = fl_filt
        print("filtered at {},{}".format(i,j))

flux_gen_filtered.to_netcdf(inDirName+processed_data_folder+pca_3+"REC_1500-modes_0_filtered.nc")